In [27]:
# Classes

class Card():
    '''A class to manage a single card'''
    
    def __init__(self,
                 value:str = None,
                 suit:str = None,
                 number:int = None):
        
        self.value = value
        self.suit = suit
        self.number = number
        self.name = self.value + ' of ' + self.suit
    
class Pile():
    '''A class to manage an ordered set of cards'''
    
    def __init__(self,
                 name:str = None,
                 card_list:list = None):
        
        self.name = name
        
        if card_list is None:
            self.card_list = []
        else:
            self.card_list = card_list
        
    def shuffle(self):
        # This method is called to shuffle the pile
        
        random.shuffle(self.card_list)
        
                
    def pile_status(self):
        # This method will print the names of cards in the pile
        
            print('\n' + self.name)
            for card in self.card_list:
                print('  ' + card.name)
                
    def sort(self,
             suit_sort:bool = False):
        # This method will sort the cards in a pile ascending, by default value then suit
        
        if suit_sort is True:
            self.card_list.sort(key=lambda x: (x.suit, x.number))
        else:
            self.card_list.sort(key=lambda x: (x.number, x.suit))
        
class Hand(Pile):
    '''A class to manage a hand of cards, child class to Pile'''
    
    def __init__(self,
                 name:str = None,
                 card_list:list = None,
                 owner:str = None):
        
        Pile.__init__(self,name,card_list)
        self.name = owner + "'s Hand"
        self.owner = owner
        
    def draw(self,
            pile:str = None,
            number:int = 1):
        # This method is called to draw a number of cards to the hand from the draw pile
        
        self.card_list.extend(pile.card_list[-number:])
        del pile.card_list[-number:]
        
    def find_book(self,
                 book_size:int = 4):
        # This method is called to identify a complete (4 cards) book of cards for the play_book method
        
        value_counter = {}
        for card in self.card_list:
            if card.value not in value_counter:
                value_counter[card.value] = 1
            else:
                value_counter[card.value] += 1
            if value_counter[card.value] == book_size:
                book_list = []
                for cardobject in self.card_list:
                    if cardobject.value == card.value:
                        book_list.append(cardobject)
                return book_list
        
        return None
        
    def play_book(self,
                  pile:str = None,
                  book:list = None,
                  scoreboard:object = None):
        # This method is called to play a book of cards from the hand to the discard pile, then update the score
        
        if book is None:
            print('\n' + self.owner + ' has no books to play.')
        else:        
            for card in book:
                self.card_list.remove(card)
                pile.card_list.append(card)
            scoreboard.update_score(player = self.owner, value = book[0].value)
            print('\n' + self.owner + ' plays the ' + book[0].value + ' book!')
            
            
    def ask(self,
            value:str = None,
            target_hand:str = None):
        # This method is used to ask for a card from another player, "Do you have any X?"
        
        print('\n' + self.owner + ' asks ' + target_hand.owner + ' if they have any ' + value + '.')
        
        card_count = 0
        new_target_list = []
        for index, card in enumerate(target_hand.card_list.copy()):
            if card.value == value:
                self.card_list.append(card)
                card_count += 1
            else:
                new_target_list.append(card)
        target_hand.card_list = new_target_list
        
        if card_count > 0:
            print('\n' + target_hand.owner + ' gives ' + str(card_count) + ' ' + value + ' to ' + self.owner + '.' )
        else:
            print('\n' + target_hand.owner + ' has no ' + value + '. ' + self.owner + ', go fish!')
        
    
class Scoreboard():
    '''A class to manage the overall game state'''
    
    def __init__(self,
                players:list = None,
                book_values:dict = None):
        
        if players is None:
            self.players = []
        else:
            self.players = players
        
        self.score = {}
        for player in self.players:
            self.score[player] = 0
        
        self.books = {}
        for key, value in book_values.items():
            self.books[value] = 0
            
    def update_score(self,
                    player:str = None,
                    value:str = None):
        # This method is called to update the score and denote a played book with value 1
        
        self.score[player] += 1
        self.books[value] = 1
    
    def print_score(self):
        # This method is called to print a summary of the game state
        
        print('\nScore  Player')
        for player in sorted(self.score, key = self.score.get, reverse = True):
            print('  ' + str(self.score[player]) + '      ' + player)
            
        played = [key for key, value in self.books.items() if value == 1]
        unplayed = [key for key, value in self.books.items() if value == 0]
        
        if len(played) > 0 and len(played) < 13:
            print('\n')
            print('Books in play:')
            for book in played:
                print('  ' + book)
        if len(unplayed) > 0:
            print('\n')
            print('Available books:')
            for book in unplayed:
                print('  ' + book)

In [28]:
# Game initialization  
import random

def startgame(player_list = ['player1','player2']):
    # Set variables and check for valid player count
    cards, hands = [], []
    player_count = len(player_list)
        
    if player_count > 1 and player_count < 4:
        starting_hand_size = 7
    elif player_count > 3 and player_count < 7:
        starting_hand_size = 5
    else:
        return print('There must be between two and six players.')
    
    suits = {0:"Clubs", 1:"Diamonds", 2:"Hearts", 3:"Spades"}
    values = {0:"Two", 1:"Three", 2:"Four", 3:"Five", 4:"Six", 5:"Seven", 6:"Eight",
              7:"Nine", 8:"Ten", 9:"Jack", 10:"Queen", 11:"King", 12:"Ace"}
    
    # Create the 52 playing cards
    for s in range(4):
        for v in range(13):
            cards.append(Card(value = values[v], suit = suits[s], number = v))
    
    # Create piles, scoreboards, and hands
    draw_pile = Pile(name = "Draw Pile", card_list = cards.copy())
    discard_pile = Pile(name = "Discard Pile")
    scoreboard = Scoreboard(players = player_list, book_values = values)
    for player in scoreboard.players:
        hands.append(Hand(owner = player))
    
    # Shuffle up and deal and sort hands
    draw_pile.shuffle()
    for hand in hands:
        hand.draw(pile = draw_pile, number = starting_hand_size)
        hand.sort()
    
    return cards, draw_pile, discard_pile, scoreboard, hands

In [40]:
#Test area
cards, draw_pile, discard_pile, scoreboard, hands = startgame(['Alex','Louis','Erin'])

for hand in hands:
    hand.pile_status()

for hand in hands:
    hand.play_book(pile = discard_pile, book = hand.find_book(book_size=3), scoreboard = scoreboard)

discard_pile.pile_status()
scoreboard.print_score()

hands[0].ask(value = 'Ace', target_hand = hands[1])

for hand in hands:
    hand.pile_status()


Alex's Hand
  Three of Spades
  Five of Clubs
  Nine of Spades
  Jack of Clubs
  King of Diamonds
  King of Hearts
  Ace of Diamonds

Louis's Hand
  Seven of Clubs
  Eight of Clubs
  Eight of Hearts
  Eight of Spades
  Ten of Clubs
  Queen of Hearts
  King of Spades

Erin's Hand
  Two of Clubs
  Two of Hearts
  Three of Clubs
  Three of Diamonds
  Jack of Diamonds
  Ace of Clubs
  Ace of Hearts

Alex has no books to play.

Louis plays the Eight book!

Erin has no books to play.

Discard Pile
  Eight of Clubs
  Eight of Hearts
  Eight of Spades

Score  Player
  1      Louis
  0      Alex
  0      Erin


Books in play:
  Eight


Available books:
  Two
  Three
  Four
  Five
  Six
  Seven
  Nine
  Ten
  Jack
  Queen
  King
  Ace

Alex asks Louis if they have any Ace.

Louis has no Ace. Alex, go fish!

Alex's Hand
  Three of Spades
  Five of Clubs
  Nine of Spades
  Jack of Clubs
  King of Diamonds
  King of Hearts
  Ace of Diamonds

Louis's Hand
  Seven of Clubs
  Ten of Clubs
  Queen of 